In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.font_manager

import os
import numpy as np
import pandas as pd


from importlib import reload

import sys
sys.path.append('../../code/scripts')
from utils import read_subset_results_nonimage
from dataset_params import dataset_params

import plotting as p

# for plotting
from matplotlib import rc
import matplotlib.font_manager


In [ ]:
# 0. setup
group_key = 'genre'
groups = [0,1]
genre_pair = ['history', 'fantasy']

#plotting
scale = 5

group_id_dict = {}
for k in range(2):
    group_id_dict[k] = genre_pair[k]
    
gamma0 = dataset_params['goodreads']['gamma']
gammas = [gamma0, 1-gamma0]
#gammas = [.37,.63]

# 1. read in results

In [ ]:
#pred_fxn_name = 'ridge'
pred_fxn_name = 'logistic_regression'
if pred_fxn_name == 'ridge':
    pred_name_print = 'ridge'
    acc_key = 'mse'
    num_seeds_eval = 10
    param_dict = {'alpha': [0.1]}
elif pred_fxn_name == 'logistic_regression':
    pred_name_print = 'logistic regression'
    acc_key = 'mae'
    param_dict = {'penalty': ['l2'], 'C': [1.0], 'solver':['lbfgs']}
    num_seeds_eval = 10
    
obj_str = 'ERM'
results_general_path = '../../results/subset_results'
results_descriptor = 'goodreads_2k_{0}_{1}_{2}_'.format(genre_pair[0],genre_pair[1], 'subsetting')

pred_fxn_base_name = 'subset_{0}'.format(group_key, pred_fxn_name)

this_results_path = os.path.join(results_general_path, results_descriptor + obj_str)
results_path_this_pred_fxn = os.path.join(this_results_path,pred_fxn_base_name, pred_fxn_name)
 
these_keys = tuple([x[0] for x in param_dict.values()])

r_ERM = read_subset_results_nonimage(results_path_this_pred_fxn,
                                           param_dict=param_dict,
                                           by_seed=True,
                                           num_seeds = num_seeds_eval,
                                           acc_keys = ['mae','mse'])[these_keys]

r_IW = read_subset_results_nonimage(results_path_this_pred_fxn.replace('ERM', 'IW'),
                                           param_dict=None,
                                           alternative_to_param_specifier='subset',
                                           by_seed=True,
                                           num_seeds = num_seeds_eval,
                                           acc_keys = ['mae','mse'])[0]


# 2. plot results

In [ ]:
# preload font sizes, etc.
reload(p)
fig, ax = p.setup_uplot_ax()

ls = ['-','--', ':']
obj_names = ['ERM', 'IW',] 
acc_key_plot = acc_key

results_to_plot = [r_ERM, r_IW,]

if acc_key == 'mae':
    gamma_annot_offest = (0.01,0.57)
elif acc_key == 'mse':
    gamma_annot_offest = (0.02,0.756)
    

for i, results in enumerate(results_to_plot):
    subset_sizes = results['subset_sizes']    
    accs_by_group = results['accs_by_group']
    accs_total = results['accs_total']
    subset_fracs = subset_sizes / subset_sizes.sum(axis=0)
    accs_total_avgs = {}

    p.plot_by_group(accs_by_group,
                    accs_total,
                    subset_fracs,
                    acc_key_plot, 
                    group_id_dict,
                    gammas = gammas,
                    pop_weights = 'by_gamma',
                    ls = ls[i],
                    label_append = obj_names[i],
                    range_type='stddev',
                    title='Goodreads: {0} model'.format(pred_name_print),
                    ylim=None,
                    plot_alpha_star=True, 
                    plot_gamma = True,
                    gamma_annot_offset = gamma_annot_offest,
                    ax = ax)
    
#ax.set_yscale('log')#loglog()
ax.grid(linewidth=1);
#ax.set_xlim(-0.01,1.01);
if acc_key == 'mae':
    ax.set_ylim(.525,.62);
    plt.ylabel(r"$\ell_1$ loss")
elif acc_key == 'mse':
    ax.set_ylim(.61,0.9)
    plt.ylabel(r"$\ell_2$ loss")

ax.set_title('Goodreads: {0} model'.format(pred_name_print), fontsize=24)
plt.savefig('../../figures/uplot_goodreads_{0}.pdf'.format(acc_key), bbox_inches='tight')

if pred_fxn_name == 'logistic_regression':
    plt.title('Goodreads rating prediction')
    plt.savefig('../../figures/uplot_goodreads_goodreads.pdf'.format(acc_key), bbox_inches='tight')
    


In [ ]:
# preload font sizes, etc.
reload(p)
fig, ax = p.setup_uplot_ax_smaller()

ls = ['-','--', ':']
obj_names = ['ERM', 'IW',] 
acc_key_plot = acc_key

results_to_plot = [r_ERM, r_IW,]

if acc_key == 'mae':
    gamma_annot_offest = (0.01,0.57)
elif acc_key == 'mse':
    gamma_annot_offest = (0.02,0.756)
    

for i, results in enumerate(results_to_plot):
    subset_sizes = results['subset_sizes']    
    accs_by_group = results['accs_by_group']
    accs_total = results['accs_total']
    subset_fracs = subset_sizes / subset_sizes.sum(axis=0)
    accs_total_avgs = {}

    p.plot_by_group(accs_by_group,
                    accs_total,
                    subset_fracs,
                    acc_key_plot, 
                    group_id_dict,
                    gammas = gammas,
                    pop_weights = 'by_gamma',
                    ls = ls[i],
                    label_append = obj_names[i],
                    range_type='stddev',
                    title='Goodreads: {0} model'.format(pred_name_print),
                    ylim=None,
                    plot_alpha_star=True, 
                    plot_gamma = True,
                    gamma_annot_offset = gamma_annot_offest,
                    group_labels_only=True,
                    ax = ax)
    
#ax.set_yscale('log')#loglog()
ax.grid(linewidth=1);
#ax.set_xlim(-0.01,1.01);
if acc_key == 'mae':
    ax.set_ylim(.525,.62);
    plt.ylabel((r"$\ell_1$ loss"))
elif acc_key == 'mse':
    ax.set_ylim(.61,0.9)
    plt.ylabel((r"$\ell_2$ loss"))

if pred_fxn_name == 'logistic_regression':
    plt.title('Goodreads rating prediction')
    plt.savefig('../../figures/uplot_goodreads_wide.pdf'.format(acc_key), bbox_inches='tight')
    